In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
from tensor_custom_core import stf_4dim, stf_4dim_time

In [3]:
num_folds = 5

In [7]:
def stf(r=2, lr=1, num_iter=100):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(num_folds=num_folds, fold_num=cur_fold)
        test_copy = test.copy()
        test_copy[:, 1:, :, :] = np.NaN
        train_test = np.concatenate([train, test_copy])
        H, A, D, T = stf_4dim(tensor=train_test,r=r,lr=lr, num_iter=num_iter )
        pred = np.einsum("Hr, Ar, Dr, Ar ->HADT", H, A, D, T)[len(train):, 1:, :, :]
        out.append(pred)
    return np.concatenate(out)


In [8]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err_stf ={}
gt = tensor[:, 1:, :, :]
for num_latent in range(2, 4):
    print(num_latent)
    pred = stf(num_latent)
    # Clamping prediction to aggregate
    pred = np.minimum(pred, tensor[:, 0:1, :,:])
    err_stf[num_latent] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

2
819.987259255 0
404.427177931 50


ValueError: einstein sum subscripts string included output subscript 'T' which never appeared in an input

In [12]:
pd.DataFrame(err_stf)

,1,2,3
dr,115.048914,115.261059,115.588028
dw,27.510341,27.287977,28.473132
fridge,35.857393,35.967953,36.028281
hvac,254.960051,232.798962,221.404030
mw,10.888660,11.028119,11.014711
